# Owner Categorization with an RNN

In this notebook, I will implement a recurrent neural network that categorize owners base on their name. Using an RNN rather than a feedfoward network is more accurate since we can include information about the *sequence* of words. Here we'll use a dataset of owners name from the Philipines and India.

The architecture for this network is shown below.

Here, we'll pass in words to an embedding layer.

From the embedding layer, the new representations will be passed to LSTM cells. These will add recurrent connections to the network so we can include information about the sequence of words in the data. Finally, the LSTM cells will go to a sigmoid output layer here. The output layer will just be a single unit then, with a sigmoid activation function.

We are not interested in the sigmoid outputs except for the very last one, we can ignore the rest. We'll calculate the cost from the output of the last step and the training label.

Charles Jansen

In [1]:
import numpy as np
import tensorflow as tf #TensorFlow 1.0
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [2]:
!pip install xlrd
fullExcel = pd.read_excel("ownersPope.xlsx")

In [3]:
fullExcel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117549 entries, 0 to 117548
Data columns (total 4 columns):
ownername                     117549 non-null object
nonPortfolioHolderTypeName    117549 non-null object
Country                       117549 non-null object
Unnamed: 3                    117549 non-null int64
dtypes: int64(1), object(3)
memory usage: 3.6+ MB


In [4]:
fullExcel.head()

,ownername,nonPortfolioHolderTypeName,Country,Unnamed: 3
0,v,Person,Pope,1
1,Na,Person,Pope,2
2,NPI,Company,Pope,3
3,GBC,Company,Pope,3
4,TIS,Company,Pope,3


In [5]:
fullExcel.ownername[232:236]

232    Sanofi
233    UPSIDC
234    SPRING
235    BIH SA
Name: ownername, dtype: object

In [6]:
names = np.array(fullExcel.ownername)
categories =  np.array(fullExcel.nonPortfolioHolderTypeName)

In [7]:
print(names[:3])
print(categories[:3])

['v' 'Na' 'NPI']
['Person' 'Person' 'Company']


## Data preprocessing

Since we're using embedding layers, we'll need to encode each word with an integer.

In [8]:
all_names = ' '.join(names)
words = all_names.split()

In [9]:
all_names[:1000]

'v Na NPI GBC TIS DIC KYB AGS DTS KEC TDK ZEN IHI KSK TPR KOA NOK TMY SMC JNC IPC JSR NTC ASO SRA HKY JCU NDS AAA DAM MMS NTN UNS NAC IIB HSM CBC DRP YSS BEC MBL HCI NBI SRI ISE Aex AMG NJS JSP TKJ JST MSS TOS HKL CDG AOI EKS GMS MMK THK AMC CIC KWS DEN ICF FJP IHN TCS TMK SHO WWG SRF SNI SMT LIC UTI SMN SUS NOB ido PTB BMS TKO GSK KDM Nil Fcb Anu Ffi Lic Fdi Sbi Uti P.c MYH KDB RAG KTB rwt NH() SM2M WEMS SCSK KDDI EKYT UACJ SBBM TOTO ATRA ACAJ IDEC JUKI UBIC CBNY EIZO MONY LINE HUMO BOZO LS.M Kazy SAFE MARU Rayo Feel GPIF Asha Mala Renu Usha Arti Abcd Fccb Amar SAMA KTB() ()AMC KAERI POSCO MOBIS TORAY Osang M-Tek Xymax MaJin LIXIL Benri KISCO UYEKI ADEKA HAYAT J・ART SAJAP NIPPO T・B・H J・M・T OKURA OKOZE ASADA K I M GAUDI TAMAX E・E・Y S-CAN JJEHD S・TEC SEIKO BOSCH HESTA TIDCO KSIDC MPEDA BANKS OHANA T-SKY i.mco TRIAD LU LI Ha Su K.KIM WU YU Ashok Swati Y.uma J Uma Deepa Apidc Shamo Banks Rahul Shama Mamta Neena Latha Pooja Rekha Kamla Nisha Divya Sonia Saroj Swamy Madan Vidya Sapna Raj S 

In [10]:
words[:20]

['v',
 'Na',
 'NPI',
 'GBC',
 'TIS',
 'DIC',
 'KYB',
 'AGS',
 'DTS',
 'KEC',
 'TDK',
 'ZEN',
 'IHI',
 'KSK',
 'TPR',
 'KOA',
 'NOK',
 'TMY',
 'SMC',
 'JNC']

### Encoding the words

The embedding lookup requires that we pass in integers to our network. The easiest way to do this is to create dictionaries that map the words in the vocabulary to integers. Then we can convert each of our owner names into integers so they can be passed into the network.

In [11]:
from collections import Counter
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii-1 for ii, word in enumerate(vocab, 1)}

names_ints = []
for each in names:
    names_ints.append([vocab_to_int[word] for word in each.split()])

### Encoding the labels (= the category here)

Our labels are "company" or "person". To use these labels in our network, we need to convert them to 0 and 1.


In [12]:
categories = np.array([1 if each == 'Company' else 0 for each in categories])

In [13]:
print(categories)

[0 0 1 ..., 1 1 1]


In [14]:
names_lens = Counter([len(x) for x in names_ints])
print("Zero-length names: {}".format(names_lens[0]))
print("Maximum name length: {}".format(max(names_lens)))

Zero-length names: 0
Maximum name length: 28


For names shorter than 28, we'll pad with 0s. 

In [15]:
seq_len = max(names_lens)
features = np.zeros((len(names_ints), seq_len), dtype=int)
for i, row in enumerate(names_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]

In [16]:
features[:10,:100]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
        53682],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
          923],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
        48125],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
        39767],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0

## Training, Validation, Test



With our data in nice shape, we'll split it into training, validation, and test sets.

10% randomly taken for test.

Kfold on the remaining 90% for validation and training.-->canceled. Done, but took much more time for the same 98.9 result

In [17]:
split_frac = 0.9
train_val_x, test_x, train_val_y, test_y = train_test_split(
    features, categories, 
    train_size = split_frac)

#sin Kfold
train_x, val_x, train_y, val_y = train_test_split(
    train_val_x, train_val_y, 
    train_size = split_frac)
'''
#Kfold
train_x = []
val_x   = []
train_y = []
val_y   = []
train_x =  np.empty([0,max(names_lens)])
val_x   =  np.empty([0,max(names_lens)])
train_y =  np.empty(0)
val_y   =  np.empty(0)

kf = KFold(n_splits = 9, shuffle=True)
for train_index, val_index in kf.split(train_val_x):
    train_temp_x, val_temp_x = train_val_x[train_index], train_val_x[val_index]
    train_temp_y, val_temp_y = train_val_y[train_index], train_val_y[val_index]
    train_x = np.concatenate((train_x, train_temp_x), axis=0)
    val_x   = np.concatenate((val_x, val_temp_x), axis=0)
    train_y = np.concatenate((train_y, train_temp_y), axis=0)
    val_y = np.concatenate((val_y, val_temp_y), axis=0)

'''
print("\t\t\tFeature Shapes:")
print("X\nTrain set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape),
      "\nY\nTrain set: \t\t{}".format(train_y.shape), 
      "\nValidation set: \t{}".format(val_y.shape),
      "\nTest set: \t\t{}".format(test_y.shape),
     )

			Feature Shapes:
X
Train set: 		(95214, 28) 
Validation set: 	(10580, 28) 
Test set: 		(11755, 28) 
Y
Train set: 		(95214,) 
Validation set: 	(10580,) 
Test set: 		(11755,)


## Build the graph

Here, we'll build the graph. First up, defining the hyperparameters.

* `lstm_size`: Number of units in the hidden layers in the LSTM cells. 
* `lstm_layers`: Number of LSTM layers in the network. I'd start with 1, then add more if I'm underfitting.
* `batch_size`: The number of names to feed the network in one training pass.
* `learning_rate`: Learning rate

In [18]:
lstm_size = 512
lstm_layers = 3
batch_size = 500
learning_rate = 0.003

For the network itself, we'll be passing in our 28 element long names vectors. Each batch will be `batch_size` vectors. We'll also be using dropout on the LSTM layer, so we'll make a placeholder for the keep probability.

In [19]:
n_words = len(vocab_to_int)

# Create the graph object
graph = tf.Graph()
# Add nodes to the graph
with graph.as_default():
    inputs_ = tf.placeholder(tf.int32, [None, None], name='inputs')
    labels_ = tf.placeholder(tf.int32, [None, None], name='labels')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

### Embedding

Now we'll add an embedding layer. 


In [20]:
len(vocab)

56699

In [21]:
vocab[:20]

['LIMITED',
 'Ltd.',
 'Co.,',
 'LTD',
 'PRIVATE',
 'Kim',
 'Limited',
 'Lee',
 'Kumar',
 'Shah',
 'PVT',
 'Ltd',
 'Association',
 'LTD.',
 '&',
 'Private',
 'Park',
 'Patel',
 'Jain',
 'Corporation']

In [22]:
# Size of the embedding vectors (number of units in the embedding layer)
embed_size = 300 

with graph.as_default():
    embedding = tf.Variable(tf.truncated_normal((n_words, embed_size), stddev=0.1))
    embed = tf.nn.embedding_lookup(embedding, inputs_)

### LSTM cell



Next, we'll create our LSTM cells to use in the recurrent network 

In [23]:
with graph.as_default():
    # Your basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    # Add dropout to the cell
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([drop] * lstm_layers)
    
    # Getting an initial state of all zeros
    initial_state = cell.zero_state(batch_size, tf.float32)

### RNN forward pass


Now we need to actually run the data through the RNN nodes. 

Above I created an initial state, `initial_state`, to pass to the RNN. This is the cell state that is passed between the hidden layers in successive time steps. 


In [24]:
with graph.as_default():
    outputs, final_state = tf.nn.dynamic_rnn(cell, embed,
                                             initial_state=initial_state)

### Output

We want the final output. So we need to grab the last output with `outputs[:, -1]`

In [25]:
with graph.as_default():
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn=tf.sigmoid)
    cost = tf.losses.mean_squared_error(labels_, predictions)
    
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

### Validation accuracy

Here we can add a few nodes to calculate the accuracy which we'll use in the validation pass.

In [26]:
with graph.as_default():
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

### Batching

This is a simple function for returning batches from our data. First it removes data such that we only have full batches. Then it iterates through the `x` and `y` arrays and returns slices out of those arrays with size `[batch_size]`.

In [27]:
def get_batches(x, y, batch_size=100):
    
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

## Training



In [28]:
'''
import os
epochs = 1

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, 2), 1):
            print(ii)
            print(x)
            print(y)
            print(x.shape)
            print(y.shape)
            print(y[:, None])
            if ii == 5:
                raise Exception("Manual Stop")
#''' ;           

In [29]:
epochs = 3

with graph.as_default():
    saver = tf.train.Saver()

In [30]:
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            feed = {inputs_: x,
                    labels_: y[:, None],
                    keep_prob: 0.5,
                    initial_state: state}
            loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)
            
            if iteration%5==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))

            if iteration%250==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed = {inputs_: x,
                            labels_: y[:, None],
                            keep_prob: 1,
                            initial_state: val_state}
                    batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
    saver.save(sess, "checkpoints/ownerNameCategJapKor.ckpt")

Epoch: 0/3 Iteration: 5 Train loss: 0.325
Epoch: 0/3 Iteration: 10 Train loss: 0.255
Epoch: 0/3 Iteration: 15 Train loss: 0.224
Epoch: 0/3 Iteration: 20 Train loss: 0.216
Epoch: 0/3 Iteration: 25 Train loss: 0.179
Epoch: 0/3 Iteration: 30 Train loss: 0.194
Epoch: 0/3 Iteration: 35 Train loss: 0.189
Epoch: 0/3 Iteration: 40 Train loss: 0.170
Epoch: 0/3 Iteration: 45 Train loss: 0.144
Epoch: 0/3 Iteration: 50 Train loss: 0.157
Epoch: 0/3 Iteration: 55 Train loss: 0.121
Epoch: 0/3 Iteration: 60 Train loss: 0.099
Epoch: 0/3 Iteration: 65 Train loss: 0.043
Epoch: 0/3 Iteration: 70 Train loss: 0.035
Epoch: 0/3 Iteration: 75 Train loss: 0.028
Epoch: 0/3 Iteration: 80 Train loss: 0.024
Epoch: 0/3 Iteration: 85 Train loss: 0.026
Epoch: 0/3 Iteration: 90 Train loss: 0.014
Epoch: 0/3 Iteration: 95 Train loss: 0.012
Epoch: 0/3 Iteration: 100 Train loss: 0.018
Epoch: 0/3 Iteration: 105 Train loss: 0.020
Epoch: 0/3 Iteration: 110 Train loss: 0.021
Epoch: 0/3 Iteration: 115 Train loss: 0.021
Epoch: 0

## Testing

In [31]:
test_acc = []
with tf.Session(graph=graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
        feed = {inputs_: x,
                labels_: y[:, None],
                keep_prob: 1,
                initial_state: test_state}
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.3f}".format(np.mean(test_acc)))

Test accuracy: 0.986


## Predictions

In [32]:
x = "Dushyant Sekhar "

x_int = [vocab_to_int[word] for word in x.split()]
x_int = x_int[:28] #ignore words after 28th

x_int_sized = np.zeros((1,seq_len), dtype=int)
x_int_sized[0,-len(x_int):] = np.array(x_int)[:seq_len]
print(x_int_sized)

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0 5087 2461]]


In [33]:
fillerSize =  batch_size - 1
filler = np.tile(np.zeros((1, seq_len), dtype=int),(fillerSize,1))
#print(filler.shape)
prodBatch = np.append(x_int_sized,filler, axis=0)
#print(prodBatch)
#print(prodBatch.shape)

In [34]:
with tf.Session(graph=graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    feed = {inputs_: prodBatch,
            keep_prob: 1,
            initial_state: test_state}
    output = sess.run(predictions, feed_dict=feed)
    print(output[0])
    if output[0]>0.5:
        print(x,"\nCompany")
        print("probability {}%".format(np.round(output[0][0]*100,2)))
    else:
        print(x,"\nPerson")
        print("probability {}%".format(np.round(100-output[0][0]*100,2)))
        

[ 0.00273275]
Dushyant Sekhar  
Person
probability 99.73%
